In [1]:
import torch
from meu_dataset import MeuDataset
from teste_util import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_dataset = "./data/datasets/features_path_flowers_dataset.pt"
# Carregar o dataset do arquivo "meu_dataset.pt"
meu_dataset = MeuDataset.load_from_file(path_dataset)
#verificar se o objeto meu dataset está retornando o tensor correto
assert isinstance(meu_dataset,MeuDataset), 'o tipo de retorno não é MeuDataset'
assert isinstance(meu_dataset[0][0],torch.Tensor), 'o tipo de retorno não é torch.Tensor'


In [2]:
import gc
from torch.optim.lr_scheduler import ExponentialLR
gc.collect()
torch.cuda.empty_cache()
batch_size_siam = 100
from torch.utils.data import random_split, DataLoader


train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.3,0.5,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=batch_size_siam, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=batch_size_siam, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size_siam, shuffle=True)

In [3]:
from torch.nn.functional import pairwise_distance
from tqdm import tqdm

def loss_fn(output_pos, output_neg, margin=15,total=100): 
    acertos_pos = torch.sum(output_pos[:, 0] == output_pos[:, 1])
    erros_neg = output_neg.shape[0]    
    loss = (total-acertos_pos+erros_neg).to(torch.float32).to(output_pos.device).detach().requires_grad_(True)
    loss = torch.relu(loss)  # Aplicar ReLU para descartar os valores negativos
    return loss/total

#Create methods to train the model
def train_one_epoch(model, data_loader, optimizer, loss_fn, device='cpu', is_training=True):
    model.train(is_training)
    total_loss = 0.
    # Definir os intervalos de colunas
    
    progress_bar = tqdm(data_loader)
    for idx,data in enumerate(progress_bar):
        #extrair as features e orientações
        batch_in,batch_out = data[0].to(device),data[1].to(device)
        print("train_one_epoch ",batch_in.shape)
        half_size = batch_in.size(0) // 2
        
        in1 = batch_in[:half_size]
        in2 = batch_in[half_size:]
        
        out1 = batch_out[:half_size]
        out2 = batch_out[half_size:]
        
        #predição no cenário positivo                
        op1 = model(in1, out1)
        op2 = model(in1, in1)
        op3 = model(in2, out2)
        op4 = model(in2, in2)
        output_pos = torch.cat([op1, op2, op3, op4], dim=0)
        #predição no cenário negativo
        on1 = model(in1, out2)
        on2 = model(in1, in2)
        on3 = model(in2, out1)
        on4 = model(in2, in1)
        output_neg = torch.cat([on1, on2, on3, on4], dim=0)
        # if idx ==0:
        #     acertos_pos = torch.sum(output_pos[:, 0] == output_pos[:, 1])
        #     erros_neg = output_neg.shape[0] 
        #     print(f'acertos_pos: {acertos_pos} - erros_neg: {erros_neg}')
            
        #calcular a loss
        loss = loss_fn(output_pos, output_neg,margin=(batch_in.size(0)*3/4),total = batch_in.size(0)*2)
        
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_description(f'Loss: {loss.item()} - Total Loss: {total_loss}')
    return total_loss

In [4]:
from siamesa_e2cnn import Siamesa
n_channel =8
PS =21
model =Siamesa(n_channel=n_channel).to(device)

/home/wagner/.local/lib/python3.11/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)


In [5]:
gc.collect()
torch.cuda.empty_cache()



def train(model,dataloader_train,dataloader_val):
    optimizer_siamese = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler = ExponentialLR(optimizer_siamese, gamma=0.75)
    best_loss = float('inf')
    best_model = None
    epochs_without_improvement = 0
    epochs = 100
    patience = 30
    
    for epoch in range(epochs):
        train_loss = train_one_epoch(model,dataloader_train , optimizer_siamese, loss_fn, device=device, is_training=True)
        with torch.no_grad():
            test_loss = train_one_epoch(model,dataloader_val , optimizer_siamese, loss_fn, device=device, is_training=False)

        # Verificar se a perda melhorou
        if test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
            best_model = model.state_dict()        
        else:
            epochs_without_improvement += 1
        
        if (epoch % 3 == 0) and (epoch != 0):
            scheduler.step()
            save_model(model, './data/models/siamese_flowers_sp.pt')
        
        # Verificar a condição de parada
        if epochs_without_improvement == patience:
            print(f"No improvement in loss for {epochs_without_improvement} epochs. Training stopped.")
            break

        print(f"Epoch [{epoch}/{epochs}] - Running Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Initial LR: {optimizer_siamese.param_groups[0]['initial_lr']:.6f}, Current LR: {optimizer_siamese.param_groups[0]['lr']:.6f}, Epochs without Improvement: {epochs_without_improvement}")

    # Carregar a melhor configuração do modelo
    model.load_state_dict(best_model)
    print(f'Epoch: {epoch}, Best Loss: {best_loss:.4f}')
    return model

model = train(model,dataloader_train,dataloader_val)

  0%|          | 0/19 [00:00<?, ?it/s]

train_one_epoch  torch.Size([100, 8, 19, 19])


Loss: 0.1899999976158142 - Total Loss: 0.1899999976158142:   5%|▌         | 1/19 [00:00<00:12,  1.47it/s]

train_one_epoch  torch.Size([100, 8, 19, 19])


Loss: 0.16500000655651093 - Total Loss: 0.35500000417232513:  11%|█         | 2/19 [00:01<00:08,  2.01it/s]

...

Loss: 0.16500000655651093 - Total Loss: 2.945000022649765:  79%|███████▉  | 15/19 [00:05<00:01,  2.65it/s]

train_one_epoch  torch.Size([100, 8, 19, 19])


Loss: 0.17000000178813934 - Total Loss: 3.1150000244379044:  84%|████████▍ | 16/19 [00:06<00:01,  2.53it/s]


train_one_epoch  torch.Size([100, 8, 19, 19])


KeyboardInterrupt: 

In [ ]:
save_model(model, './data/models/siamese_flowers_sp.pt')
model =Siamesa(n_channel=n_channel)
model.to(device)
load_model(model, './data/models/siamese_flowers_sp.pt',device)

# Senao treinar pular para cá

In [ ]:
import numpy as np
def matching_single_points_all_dataloader(model,data_loader,device):
    total_acertos = 0
    total =0
    tqdm_loader = tqdm(data_loader)
    for data1,data2 in tqdm_loader:
        size = data2.shape[0]
        matches = []
        model.eval()
        with torch.no_grad():
            matches = model(data1.to(device),data2.to(device))
            matches = np.array(matches)
            acertos = np.sum(matches[:, 0] == matches[:, 1])
            total_acertos += acertos
            total += len(matches)
            print(acertos,total_acertos,total)
            perc_acertos = total_acertos*100/total
            tqdm_loader.set_description(f"perc_acertos: {perc_acertos:.2f}  total: {total_acertos}/{total}")
    return total_acertos,total

torch.manual_seed(0)
torch.cuda.manual_seed(0)
matching_single_points_all_dataloader(model,dataloader_val,device)